In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, cohen_kappa_score
from scipy.signal import resample
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Load datasets
eeg_data = pd.read_csv('eeg_data_trimmed.csv')
emg_data = pd.read_csv('emg_data_trimmed.csv')
labels = pd.read_csv('labels_trimmed.csv')

# Label encoding
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels.values.ravel())

# Ensure combined_data and labels are aligned
if len(labels) > len(eeg_data):
    labels = labels[:len(eeg_data)]
elif len(labels) < len(eeg_data):
    eeg_data = eeg_data.iloc[:len(labels)]
    emg_data = emg_data.iloc[:len(labels)]

# Resample EEG to match EMG's length (if necessary)
if len(eeg_data) != len(emg_data):
    eeg_data = resample(eeg_data, num=len(emg_data))

# Dynamic Time Warping (DTW) for offline alignment
distance, path = fastdtw(eeg_data.values, emg_data.values, dist=euclidean)
aligned_eeg = eeg_data.iloc[[p[0] for p in path]]
aligned_emg = emg_data.iloc[[p[1] for p in path]]

# Combine aligned data
combined_data = pd.concat([aligned_eeg.reset_index(drop=True), aligned_emg.reset_index(drop=True)], axis=1)

# Ensure combined_data and labels are still aligned
if len(labels) > len(combined_data):
    labels = labels[:len(combined_data)]
elif len(labels) < len(combined_data):
    combined_data = combined_data.iloc[:len(labels)]

class EEGEMGDataset(Dataset):
    def __init__(self, X, y, noise_std=0.01):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
        self.noise_std = noise_std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X_noisy = self.X[idx] + torch.normal(0, self.noise_std, size=self.X[idx].shape)
        return X_noisy, self.y[idx]

# CNN-GRU Model
class CNNGRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, kernel_size=3, num_filters=16, dropout_rate=0.7):
        super(CNNGRUModel, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=kernel_size, padding=kernel_size // 2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.gru = nn.GRU(input_size=num_filters, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  
        cnn_out = self.cnn(x).permute(0, 2, 1)  
        gru_out, _ = self.gru(cnn_out)
        output = self.fc(gru_out[:, -1, :])  
        return output

# Train function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=25, patience=5, model_save_path="best_model.pth"):
    best_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        val_loss = 0
        model.eval()
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}, Val Loss: {val_loss:.4f}")

        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict()
            torch.save(best_model_state, model_save_path)  
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                model.load_state_dict(best_model_state)
                break

# Compute Metrics Function (FIX)
def compute_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    return precision, recall, f1, accuracy

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(y_batch.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    precision, recall, f1, accuracy = compute_metrics(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    kappa_score = cohen_kappa_score(y_true, y_pred)

    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}, Accuracy: {accuracy:.2f}")
    print(f"Cohen's Kappa: {kappa_score:.2f}")
    print("Confusion Matrix:")
    print(conf_matrix)

    return precision, recall, f1, accuracy, kappa_score, conf_matrix

# Online Learning Function
def online_adaptation(model, optimizer, buffer_X, buffer_y, criterion, val_loader, num_cycles=3, batch_size=16, lr=0.0001):
    model.train()
    buffer_dataset = EEGEMGDataset(buffer_X, buffer_y)
    buffer_loader = DataLoader(buffer_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.005)

    for cycle in range(num_cycles):
        print(f"Online Adaptation Cycle {cycle + 1}/{num_cycles}")
        for X_batch, y_batch in buffer_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        print("Validation Metrics After Cycle:")
        evaluate_model(model, val_loader)

# K-Fold Cross-Validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
fold_results = []
online_adaptation_percentage = 0.3  

for fold, (train_index, val_index) in enumerate(kf.split(combined_data)):
    print(f"Fold {fold + 1}/{k}")
    X_train, X_val = combined_data.iloc[train_index].values, combined_data.iloc[val_index].values
    y_train, y_val = labels[train_index], labels[val_index]

    adaptation_size = int(len(X_val) * online_adaptation_percentage)
    buffer_X, buffer_y = X_val[:adaptation_size], y_val[:adaptation_size]
    X_val, y_val = X_val[adaptation_size:], y_val[adaptation_size:]

    train_dataset = EEGEMGDataset(X_train, y_train, noise_std=0.02)
    val_dataset = EEGEMGDataset(X_val, y_val, noise_std=0.0)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    model = CNNGRUModel(input_dim=X_train.shape[1], hidden_dim=128, num_classes=len(np.unique(labels)), num_filters=16, dropout_rate=0.7)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

    train_model(model, train_loader, val_loader, criterion, optimizer)
    
    print("\nInitial Evaluation Before Online Learning:")
    evaluate_model(model, val_loader)
    
    online_adaptation(model, optimizer, buffer_X, buffer_y, criterion, val_loader)
    
    print("\nFinal Evaluation After Online Learning:")
    evaluate_model(model, val_loader)


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, cohen_kappa_score
from scipy.signal import resample
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Load datasets
eeg_data = pd.read_csv('eeg_data_trimmed.csv')
emg_data = pd.read_csv('emg_data_trimmed.csv')
labels = pd.read_csv('labels_trimmed.csv')

# Label encoding
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels.values.ravel())

# Ensure combined_data and labels are aligned
if len(labels) > len(eeg_data):
    labels = labels[:len(eeg_data)]
elif len(labels) < len(eeg_data):
    eeg_data = eeg_data.iloc[:len(labels)]
    emg_data = emg_data.iloc[:len(labels)]

# Resample EEG to match EMG's length (if necessary)
if len(eeg_data) != len(emg_data):
    eeg_data = resample(eeg_data, num=len(emg_data))

# Dynamic Time Warping (DTW) for offline alignment
distance, path = fastdtw(eeg_data.values, emg_data.values, dist=euclidean)
aligned_eeg = eeg_data.iloc[[p[0] for p in path]]
aligned_emg = emg_data.iloc[[p[1] for p in path]]

# Combine aligned data
combined_data = pd.concat([aligned_eeg.reset_index(drop=True), aligned_emg.reset_index(drop=True)], axis=1)

# Ensure combined_data and labels are still aligned
if len(labels) > len(combined_data):
    labels = labels[:len(combined_data)]
elif len(labels) < len(combined_data):
    combined_data = combined_data.iloc[:len(labels)]

class EEGEMGDataset(Dataset):
    def __init__(self, X, y, noise_std=0.002):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
        self.noise_std = noise_std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X_noisy = self.X[idx] + torch.normal(0, self.noise_std, size=self.X[idx].shape)
        return X_noisy, self.y[idx]

# CNN-RNN Model
class CNNRNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_filters=16, dropout_rate=0.7):
        super(CNNRNNModel, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.rnn = nn.RNN(input_size=num_filters, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        x = x.unsqueeze(1)
        cnn_out = self.cnn(x).permute(0, 2, 1)
        rnn_out, _ = self.rnn(cnn_out)
        output = self.fc(rnn_out[:, -1, :])
        return output

# TCN Model
class TCNModel(nn.Module):
    def __init__(self, input_dim, num_classes, num_filters=16, dropout_rate=0.7):
        super(TCNModel, self).__init__()
        self.tcn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=num_filters, out_channels=num_filters, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate)
        )
        self.fc = nn.Linear(num_filters, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        tcn_out = self.tcn(x).mean(dim=2)
        output = self.fc(tcn_out)
        return output

# Online Adaptation Function
def online_adaptation(model, optimizer, buffer_X, buffer_y, criterion, val_loader, num_cycles=3, batch_size=16, lr=0.0001):
    model.train()
    buffer_dataset = EEGEMGDataset(buffer_X, buffer_y)
    buffer_loader = DataLoader(buffer_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.005)

    for cycle in range(num_cycles):
        print(f"Online Adaptation Cycle {cycle + 1}/{num_cycles}")
        for X_batch, y_batch in buffer_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        print("Validation Metrics After Cycle:")
        evaluate_model(model, val_loader)

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(y_batch.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    precision, recall, f1, accuracy = compute_metrics(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    kappa_score = cohen_kappa_score(y_true, y_pred)

    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}, Accuracy: {accuracy:.2f}")
    print(f"Cohen's Kappa: {kappa_score:.2f}")
    print("Confusion Matrix:")
    print(conf_matrix)

    return precision, recall, f1, accuracy, kappa_score, conf_matrix

# k-Fold Cross-Validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
online_adaptation_percentage = 0.3  

for fold, (train_index, val_index) in enumerate(kf.split(combined_data)):
    print(f"Fold {fold + 1}/{k}")
    X_train, X_val = combined_data.iloc[train_index].values, combined_data.iloc[val_index].values
    y_train, y_val = labels[train_index], labels[val_index]

    adaptation_size = int(len(X_val) * online_adaptation_percentage)
    buffer_X, buffer_y = X_val[:adaptation_size], y_val[:adaptation_size]
    X_val, y_val = X_val[adaptation_size:], y_val[adaptation_size:]

    train_dataset = EEGEMGDataset(X_train, y_train, noise_std=0.002)
    val_dataset = EEGEMGDataset(X_val, y_val, noise_std=0.0)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Select Model: CNN-RNN or TCN
    model = CNNRNNModel(input_dim=X_train.shape[1], hidden_dim=128, num_classes=len(np.unique(labels)), num_filters=16, dropout_rate=0.7)
    # model = TCNModel(input_dim=X_train.shape[1], num_classes=len(np.unique(labels)), num_filters=16, dropout_rate=0.7)  # Uncomment for TCN

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

    train_model(model, train_loader, val_loader, criterion, optimizer)
    
    print("\nInitial Evaluation Before Online Learning:")
    evaluate_model(model, val_loader)
    
    online_adaptation(model, optimizer, buffer_X, buffer_y, criterion, val_loader)
    
    print("\nFinal Evaluation After Online Learning:")
    evaluate_model(model, val_loader)


In [ ]:

for fold, (train_index, val_index) in enumerate(kf.split(combined_data)):
    print(f"Fold {fold + 1}/{k}")
    X_train, X_val = combined_data.iloc[train_index].values, combined_data.iloc[val_index].values
    y_train, y_val = labels[train_index], labels[val_index]

    adaptation_size = int(len(X_val) * online_adaptation_percentage)
    buffer_X, buffer_y = X_val[:adaptation_size], y_val[:adaptation_size]
    X_val, y_val = X_val[adaptation_size:], y_val[adaptation_size:]

    train_dataset = EEGEMGDataset(X_train, y_train, noise_std=0.002)
    val_dataset = EEGEMGDataset(X_val, y_val, noise_std=0.0)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Select Model: CNN-RNN or TCN
    #model = CNNRNNModel(input_dim=X_train.shape[1], hidden_dim=128, num_classes=len(np.unique(labels)), num_filters=16, dropout_rate=0.7)
    model = TCNModel(input_dim=X_train.shape[1], num_classes=len(np.unique(labels)), num_filters=16, dropout_rate=0.7)  # Uncomment for TCN

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

    train_model(model, train_loader, val_loader, criterion, optimizer)
    
    print("\nInitial Evaluation Before Online Learning:")
    evaluate_model(model, val_loader)
    
    online_adaptation(model, optimizer, buffer_X, buffer_y, criterion, val_loader)
    
    print("\nFinal Evaluation After Online Learning:")
    evaluate_model(model, val_loader)
